In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import FactorAnalysis
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.metrics import mean_squared_error, max_error, mean_gamma_deviance, r2_score
from pandas.plotting import scatter_matrix
from matplotlib import pyplot as plt


In [ ]:
X = pd.read_csv('5_x.csv', header=None)
X.head()

In [ ]:
scatter_matrix(X)
X.corr()

### Фичи не коррелируют, что делать дальше я не понял

In [ ]:
y = pd.read_csv('5_y.csv', header=None)

In [ ]:
clf = LinearRegression(normalize=True, n_jobs=-1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.coef_

In [ ]:
clf.score(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
def print_predicted(predicted, real):
    print(f"Предсказанное - настоящее")
    for a, b in np.squeeze(np.dstack((predicted, real))):
        print(f"{a:>13f} - {b:<=6f}")

In [ ]:
print_predicted(clf.predict(X_train.iloc[0:10]), y_train.iloc[0:10])

In [ ]:
mean_squared_error(y_train.iloc[0:10], clf.predict(X_train.iloc[0:10]))

In [ ]:
print_predicted(clf.predict(X_test.iloc[:10]), y_test.iloc[:10])

In [ ]:
mean_squared_error(y_test.iloc[0:10], clf.predict(X_test.iloc[0:10]))

#### Сделаем из фичей множесво полиномиальных комбинаций
https://habr.com/ru/post/414245/

In [ ]:
p = PolynomialFeatures(degree=4, order='F')
X_p = p.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_p, y, test_size=0.2, random_state=123)

In [ ]:
clf = LinearRegression(n_jobs=-1)
clf.fit(X_train,y_train)

In [ ]:
clf.score(X_train, y_train)

In [ ]:
print(f"{clf.score(X_train, y_train)=}\n{clf.score(X_test, y_test)=}")

In [ ]:
print_predicted(clf.predict(X_train[:10]), y_train[:10])

In [ ]:
print_predicted(clf.predict(X_test[:10]), y_test[:10])

Получилось что-то странное

#### Lasso
С помощью GridSearchCV подберем лучшие параметры

In [ ]:
clf = Lasso()
gscv = GridSearchCV(clf,
                    {'alpha': [i / 10 for i in range(1, 11)],
                    'normalize': [True, False],
                    'max_iter': [i * 100 for i in range(1, 15)],
                    'tol': [i / 10000 for i in range(1, 10)]},
                    cv=5, n_jobs=-1, verbose=1)


In [ ]:
gscv.fit(X_train, y_train)

In [ ]:
gscv.best_params_

In [ ]:
gscv.score(X_train, y_train)

In [ ]:
gscv.score(X_test, y_test)

In [ ]:
X_gscv_predict = gscv.predict(X_train[:10])

In [ ]:
mean_squared_error(y_train[:10], X_gscv_predict)


In [ ]:
X_gscv_predict_test = gscv.predict(X_test[:10])
X_gscv_predict_test

In [ ]:
mean_squared_error(y_test[:10], X_gscv_predict_test)

Ну оно отработало получше

# Big Data 1

In [ ]:
X_big = pd.read_csv('1_x.csv', header=None)
y_big = pd.read_csv('1_y.csv', header=None)
X_big.head()

#### По ощущениям это очень долго...

In [ ]:
# scatter_matrix(X_big)

In [ ]:
c = X_big.corr().abs()

In [ ]:
c.unstack().sort_values(ascending = False).drop_duplicates()

#### Не похоже, что тут с корреляцией все плохо

In [ ]:
X_train_big, X_test_big, y_train_big, y_test_big = train_test_split(X_big, y_big, test_size=0.2, random_state=123)

In [ ]:
clf_big = Lasso()
gscv_big = GridSearchCV(clf_big, {'alpha': [0.1], 'max_iter': [100], 'normalize': [True, False], 'tol': [0.0001]},
                    cv=5, n_jobs=-1, verbose=1, return_train_score=True)

#### Параметры посчитал сам, ибо они долго подбираются :/

In [ ]:
gscv_big.fit(X_train_big, y_train_big)

In [ ]:
gscv_big.best_params_

In [ ]:
gscv_big.score(X_train_big, y_train_big)

In [ ]:
gscv_big.score(X_test_big, y_test_big)

In [ ]:
X_gscv_big_predict = gscv_big.predict(X_train_big[:40])

In [ ]:
mean_squared_error(y_train_big[:40], X_gscv_big_predict)

In [ ]:
X_gscv_big_predict_test = gscv_big.predict(X_test_big[:40])
X_gscv_big_predict_test

In [ ]:
mean_squared_error(y_test_big[:40], X_gscv_big_predict_test)

In [ ]:
r2_score(y_test_big[:40], X_gscv_big_predict_test)

In [ ]:
print_predicted(X_gscv_big_predict_test, y_test_big[:40][0].tolist())

#### Выглядит хорошо :/
